# Koog



[Koog](https://docs.koog.ai/) a multiplatform library for building AI empowered applications in idiomatic Kotlin (in a way that feels natural for Kotlin devs).
It supports state of the art features like agents, MCP, tools, embeddings, and more.
It can currently target the JVM, WASM and JS.

Since it supports the JVM, we can use it in a Kotlin notebook.
Let's start by adding the dependency.

In [6]:
%use koog(0.3.0)

## Single-run agents

Create an agent for different providers.


In [2]:
val systemPrompt = "You are a helpful assistant. Answer user questions concisely."

fun getOpenAiAgent() = AIAgent(
  executor = simpleOpenAIExecutor(System.getenv("OPENAI_API_KEY")),
  systemPrompt = systemPrompt,
  llmModel = OpenAIModels.Chat.GPT4o
)

fun getGoogleAiAgent() = AIAgent(
  executor = simpleGoogleAIExecutor(System.getenv("GOOGLEAI_API_KEY")),
  systemPrompt = systemPrompt,
  llmModel = GoogleModels.Gemini2_0FlashLite
)

You can get a free google AI API key from [Google AI Studio](https://aistudio.google.com/apikey).
Please add to your user environment variables as `GOOGLEAI_API_KEY`, and then reboot your IDE (so that is loads the new environment variable).

Uncomment the agent you want to use.

In [4]:
//val agent = getOpenAiAgent()
val agent = getGoogleAiAgent()

In [5]:
import kotlinx.coroutines.runBlocking

runBlocking {
  val result = agent.run("Hello! How can you help me?")
  println(result)
}

I can help you with a variety of tasks, such as answering questions, summarizing text, generating creative content, and more. Just let me know what you need!



## Tool calling

First, define the toolset

In [7]:
import ai.koog.agents.core.tools.annotations.Tool

@LLMDescription("Tools for performing basic arithmetic operations")
class CircleGeometryToolset : ToolSet {
  @Tool
  @LLMDescription("Circle area calculation")
  fun area(
    @LLMDescription("Circle radius (floating point value)")
    r: Double,
  ) = r * r * Math.PI

  @Tool
  @LLMDescription("Circle circumference calculation")
  fun circumference(
    @LLMDescription("Circle radius (floating point value)")
    r: Double,
  ) = r * 2 * Math.PI
}

// Add the tool to the tool registry
val toolRegistry = ToolRegistry {
  tools(CircleGeometryToolset())
}

Add an event handler feature to log tool calls

In [8]:
import ai.koog.agents.features.eventHandler.feature.EventHandler

val features: AIAgent.FeatureContext.() -> Unit = {
  install(EventHandler) {
    onToolCall { println("tool called $it") }
  }
}

Define the agent

In [14]:
val circleGeomertrySystemPrompt =
  "You are a helpful assistant that can perform circle area and circumference calculations using the provided tools. Please show 2 decimal points in your final answer."

fun getGoogleAiAgentWithTools() = AIAgent(
  executor = simpleGoogleAIExecutor(System.getenv("GOOGLEAI_API_KEY")),
  llmModel = GoogleModels.Gemini2_0FlashLite,
  systemPrompt = circleGeomertrySystemPrompt,
  toolRegistry = toolRegistry,
  installFeatures = features
)

fun getOpenAiAgentWithTools() = AIAgent(
  executor = simpleOpenAIExecutor(System.getenv("OPENAI_API_KEY")),
  llmModel = OpenAIModels.Chat.GPT4o,
  systemPrompt = circleGeomertrySystemPrompt,
  toolRegistry = toolRegistry,
  installFeatures = features
)

val agentWithTools = getGoogleAiAgentWithTools()
//val agentWithTools = getOpenAiAgentWithTools()

In [15]:
import kotlinx.coroutines.runBlocking

runBlocking {
  val result = agentWithTools.run("What is the area and circumference of a circle with radius 5?")
  println(result)
  println("Manually computed area: ${(25 * Math.PI)} Circumference: ${5 * 2 * Math.PI}")
}

tool called ToolCallContext(runId=44f8c993-9791-45f6-a95f-aef5330a0700, toolCallId=fdd36998-17ff-4166-9882-cd764893c3e9, tool=ai.koog.agents.core.tools.reflect.ToolFromCallable@3e0c5b7f, toolArgs=VarArgs(args={parameter #1 r of fun Line_8_jupyter.CircleGeometryToolset.area(kotlin.Double): kotlin.Double=5.0}))
tool called ToolCallContext(runId=44f8c993-9791-45f6-a95f-aef5330a0700, toolCallId=2df07d46-cf80-4d53-832e-4dc04ea545d7, tool=ai.koog.agents.core.tools.reflect.ToolFromCallable@5af48146, toolArgs=VarArgs(args={parameter #1 r of fun Line_8_jupyter.CircleGeometryToolset.circumference(kotlin.Double): kotlin.Double=5.0}))
The area of the circle is 78.54 and the circumference is 31.42.

Manually computed area: 78.53981633974483 Circumference: 31.41592653589793
